# Castle Pavers - Production Scheduling

In [1]:
from pulp import *

In [2]:
NumMonths = 3
Months = [t+1 for t in range(NumMonths)]

In [3]:
StoneType = ['Colonial', 'Modern']
InitialInventory = {'Colonial': 0, 'Modern': 0}

In [4]:
Demand = {('Colonial',1): 40, ('Colonial',2): 30, ('Colonial',3): 20,
          ('Modern',1): 20, ('Modern',2): 30, ('Modern',3): 10}
ProdCost = {('Colonial',1): 300, ('Colonial',2): 340, ('Colonial',3): 380,
          ('Modern',1): 250, ('Modern',2): 280, ('Modern',3): 340}
InvCost = {'Colonial':50, 'Modern':40}
Capacity = 65

In [5]:
Prob = LpProblem("Castle Pavers", LpMinimize)

In [6]:
Production = LpVariable.dicts("Production",(StoneType,Months),0)
Inventory  = LpVariable.dicts("Inventory" ,(StoneType,Months),0)

In [7]:
Prob += lpSum([ProdCost[(i,t)]*Production[i][t] + InvCost[i]*Inventory[i][t] for i in StoneType for t in Months]), "Total Cost"

AllMonthsButTheFirst = Months[1:NumMonths]
for i in StoneType:
    Prob += lpSum([InitialInventory[i] + Production[i][1] - Inventory[i][1]]) == Demand[(i,1)], "Balance_\%s_1"%i
    for t in AllMonthsButTheFirst:
        Prob += lpSum([Inventory[i][t-1] + Production[i][t] - Inventory[i][t]]) == Demand[(i,t)], "Balance_%s_\%s"%(i,t)
        
for t in Months:
    Prob += lpSum([Production[i][t] for i in StoneType]) <= Capacity, "Production_Capacity_%s"%t

In [8]:
Prob.writeLP("CastlePavers.lp")
Prob.solve()
print("Status:", LpStatus[Prob.status])

Status: Optimal


In [9]:
for variable in Prob.variables():
    print ("{}* = {}".format(variable.name, variable.varValue))

print("Optimal Cost=", "$"+ str(value(Prob.objective)))

Inventory_Colonial_1* = 5.0
Inventory_Colonial_2* = 0.0
Inventory_Colonial_3* = 0.0
Inventory_Modern_1* = 0.0
Inventory_Modern_2* = 10.0
Inventory_Modern_3* = 0.0
Production_Colonial_1* = 45.0
Production_Colonial_2* = 25.0
Production_Colonial_3* = 20.0
Production_Modern_1* = 20.0
Production_Modern_2* = 40.0
Production_Modern_3* = 0.0
Optimal Cost= $46450.0
